# Plot ROI TFRs: 

- For each ROI in SWB dataset - plot within subj ROI averaged TFRs by condition + across subj ROI TFRs by condition


*Created: 04/08/24* \
*Updated: 06/24/2024*


In [1]:
import numpy as np
import mne
from glob import glob
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from scipy.stats import zscore, linregress, ttest_ind, ttest_rel, ttest_1samp, pearsonr, spearmanr
import pandas as pd
from mne.preprocessing.bads import _find_outliers
import os 
import joblib
import re
import datetime
import scipy
import random
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.regression.mixed_linear_model import MixedLM
import pickle
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('/sc/arion/projects/guLab/Alie/SWB/ephys_analysis/LFPAnalysis/')

In [4]:
from LFPAnalysis import lfp_preprocess_utils, sync_utils, analysis_utils, nlx_utils

In [5]:
# Specify root directory for un-archived data and results 
base_dir      = '/sc/arion/projects/guLab/Alie/SWB/'
anat_dir      = f'{base_dir}ephys_analysis/recon_labels/'
neural_dir    = f'{base_dir}ephys_analysis/data/'
behav_dir     = f'{base_dir}swb_behav_models/data/behavior_preprocessed/'
save_dir      = f'{base_dir}ephys_analysis/results/'
subj_info_dir = f'{base_dir}ephys_analysis/subj_info/'

date = datetime.date.today().strftime('%m%d%Y')
print(date)

06242024


In [6]:
# set plot themes and global plot vars 
sns.set_context('poster')
freqs = np.logspace(*np.log10([2, 200]), num=30) # set frequencies 
times = np.round(list(np.arange(0.0,3.002,0.002)),4)



In [7]:
# Subj list for CCN 2024
subj_ids = list(pd.read_excel(f'{subj_info_dir}SWB_subjects.xlsx', sheet_name='Usable_Subjects', usecols=[0]).PatientID)
n_subj = len(subj_ids)

bdi_list = pd.read_excel(f'{base_dir}ephys_analysis/subj_info/SWB_subjects.xlsx', sheet_name='Usable_Subjects', usecols=[3])
bdi_list = list(bdi_list.SWB_BDI)
subj_info_df = pd.DataFrame({'subj_id':subj_ids,'bdi':bdi_list})
subj_info_df



,subj_id,bdi
0,MS002,14
1,MS003,8
2,MS009,16
3,MS011,13
4,MS015,26
5,MS016,10
6,MS017,26
7,MS019,12
8,MS022,10
9,MS024,16


## Load all subj anat info + threshold ROIs by count

In [8]:
roi_reref_labels = pd.read_csv(f'{save_dir}roi_info/roi_reref_labels_master.csv')
roi_reref_labels = roi_reref_labels.drop(columns=['Unnamed: 0'])
roi_reref_labels

,subj_id,reref_ch_names,ch_label4roi,ch_type4roi,loc4roi,mni_x,mni_y,mni_z,hemi,roi
0,MS002,lacas1-lacas2,lacas1,anode,left cingulate gyrus d,-6.382462,37.158688,-3.130044,l,acc
1,MS002,lacas2-lacas3,lacas2,anode,left cingulate gyrus e,-6.368174,38.606223,2.270621,l,acc
2,MS002,lacas3-lacas4,lacas3,anode,left cingulate gyrus f,-6.390079,39.941566,7.640265,l,acc
3,MS002,lacas4-lacas5,lacas4,anode,left cingulate gyrus f,-6.914519,41.546899,12.993427,l,acc
4,MS002,lacas5-lacas6,lacas5,anode,left cingulate gyrus g,-6.933604,42.745784,18.267675,l,acc
...,...,...,...,...,...,...,...,...,...,...
1875,DA039,rtp6-rtp7,rtp6,anode,right superior middle temporal pole d,40.532268,7.354525,-33.253436,r,temporal pole
1876,DA039,rtp7-rtp8,rtp7,anode,right superior middle temporal pole d,43.911226,7.341369,-31.915085,r,temporal pole
1877,DA039,rsgcc6-rsgcc7,rsgcc7,cathode,right anterior pars triangularis b,42.378987,31.277597,2.471854,r,vlpfc
1878,DA039,rsgcc7-rsgcc8,rsgcc7,anode,right anterior pars triangularis b,42.378987,31.277597,2.471854,r,vlpfc


In [9]:
roi_reref_labels['roi'].value_counts()


dmpfc            216
ofc              202
acc              193
sts              155
hpc              135
amy              113
dlpfc            107
stg              105
vlpfc             82
ains              71
smg               66
pins              60
thalamus          58
mtg               57
temporal pole     31
mcc               28
motor             25
phg               25
parietal          24
vmpfc             23
fusiform          19
occipital         19
itg               15
angular           14
somat             11
premotor           9
caudate            8
pcc                1
Name: roi, dtype: int64

In [10]:
roi_list = roi_reref_labels.roi.value_counts().index[roi_reref_labels.roi.value_counts().values>=20].tolist()
roi_list

['dmpfc',
 'ofc',
 'acc',
 'sts',
 'hpc',
 'amy',
 'dlpfc',
 'stg',
 'vlpfc',
 'ains',
 'smg',
 'pins',
 'thalamus',
 'mtg',
 'temporal pole',
 'mcc',
 'motor',
 'phg',
 'parietal',
 'vmpfc']

## Load all subj behav + clean 

In [11]:
#### load all subj behav data 
all_behav = []

for subj_id in subj_ids:
    subj_df         = pd.read_csv(f'{behav_dir}{subj_id}_task_data')
    rounds_t1       = subj_df.Round.tolist()[1:]
    logRT_list      = np.log(subj_df['RT']).tolist() # take log of RT
    logRT_t1_list   = logRT_list[1:] # get logRT_t1
    GambleChoice_t1 = subj_df.GambleChoice.tolist()[1:]
    GambleEV_t1     = subj_df.GambleEV.tolist()[1:]
    TrialType_t1    = subj_df.TrialType.tolist()[1:]
    TrialEV         = subj_df.GambleEV - subj_df.SafeBet
    TrialEV_t1      = TrialEV[1:].tolist()
    rounds_t1.append(np.nan)
    logRT_t1_list.append(np.nan) # add nan to logRT_t1 (no t1 for last round)
    GambleChoice_t1.append(np.nan)
    GambleEV_t1.append(np.nan)
    TrialType_t1.append(np.nan) 
    TrialEV_t1.append(np.nan)
    subj_df['logRT']           = logRT_list
    subj_df['logRT_t1']        = logRT_t1_list
    subj_df['Round_t1']        = rounds_t1
    subj_df['GambleChoice_t1'] = GambleChoice_t1
    subj_df['GambleEV_t1']     = GambleEV_t1
    subj_df['TrialType_t1']    = TrialType_t1
    subj_df['TrialEV']         = TrialEV
    subj_df['TrialEV_t1']      = TrialEV_t1
    subj_df['bdi']             = subj_info_df.bdi[subj_info_df.subj_id==subj_id].values[0]
    subj_df['subj_id']         = subj_id

    # remove trials where RT is too fast (random choice)
    subj_df = subj_df[subj_df.RT>0.3].reset_index(drop=True)
    # remove trials where there is no choice at trial t
    subj_df = subj_df[(subj_df.GambleChoice=='gamble') | 
                      (subj_df.GambleChoice == 'safe')].reset_index(drop=True)

    # remove trials where there is no outcome
    subj_df = subj_df[(subj_df.Outcome=='good') | 
                                            (subj_df.Outcome == 'bad')].reset_index(drop=True)
    # remove epoch 76 bc photodiode times are not correct at breakpoint 
    subj_df = subj_df[subj_df.Round != 76].reset_index(drop=True)
    subj_df = subj_df[~np.isinf(subj_df.logRT_t1)].reset_index(drop=True)
    # format GambleChoice + TrialType vars as categories 
    subj_df['GambleChoice']    = subj_df['GambleChoice'].astype('category')
    subj_df['GambleChoice_t1'] = subj_df['GambleChoice_t1'].astype('category')
    subj_df['TrialType']       = subj_df['TrialType'].astype('category')
    subj_df['TrialType_t1']    = subj_df['TrialType_t1'].astype('category')
    # make sure Round_t1 is an integer
    subj_df['Round_t1']        = subj_df['Round_t1'].astype('Int64')
    
#   # zscore continuous regressors AFTER removing bad trials!! 
#     for reg in continuous_regressors:
# #         subj_regression_df[reg] = norm_zscore(subj_regression_df[reg].values) # zscore is /2std
# continuous_regressors = ['TrialEV','TrialEV_t1','GambleEV','GambleEV_t1','SafeBet',
#                          'TotalProfit','CR', 'choiceEV', 'RPE','Profit',
#                          'decisionCPE', 'decisionRegret','decisionRelief']

    all_behav.append(subj_df)

all_behav = pd.concat(all_behav).reset_index(drop=True)
all_behav.to_csv(f'{save_dir}roi_tfr_data.csv')

In [12]:
all_behav

,Round,Trial Num,TrialType,TrialOnset,ChoiceOnset,DecisionOnset,FeedbackOnset,RT,SafeBet,LowBet,...,logRT,logRT_t1,Round_t1,GambleChoice_t1,GambleEV_t1,TrialType_t1,TrialEV,TrialEV_t1,bdi,subj_id
0,1,25.0,mix,513.380590,513.390239,515.450091,515.457173,2.059852,0.0,-1.10,...,0.722634,0.670167,2,gamble,-0.475,loss,-0.265,-0.175,14,MS002
1,2,117.0,loss,522.640856,522.641563,524.596127,526.627092,1.954564,-0.3,-0.95,...,0.670167,0.459614,3,gamble,0.840,gain,-0.175,0.440,14,MS002
2,3,79.0,gain,531.174799,531.175599,532.759061,534.780269,1.583462,0.4,0.00,...,0.459614,0.912929,4,safe,0.200,mix,0.440,0.200,14,MS002
3,4,42.0,mix,545.592613,545.593355,548.084966,548.092333,2.491611,0.0,-0.80,...,0.912929,0.570378,5,gamble,0.620,gain,0.200,0.120,14,MS002
4,5,85.0,gain,555.337336,555.345720,557.114656,559.135069,1.768936,0.5,0.00,...,0.570378,0.631657,6,safe,-0.185,mix,0.120,-0.185,14,MS002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3739,146,79.0,gain,2259.827656,2259.828749,2260.908450,2262.926195,1.079701,0.4,0.00,...,0.076684,0.608282,147,safe,-0.450,mix,0.440,-0.450,22,DA039
3740,147,30.0,mix,2267.502359,2267.534059,2269.371331,2269.377701,1.837272,0.0,-1.50,...,0.608282,1.393768,148,safe,-0.045,mix,-0.450,-0.045,22,DA039
3741,148,13.0,mix,2282.349445,2282.350662,2286.380667,2286.389886,4.030006,0.0,-0.50,...,1.393768,1.152830,149,safe,-0.190,mix,-0.045,-0.190,22,DA039
3742,149,18.0,mix,2293.040983,2293.042042,2296.209186,2296.218136,3.167144,0.0,-0.80,...,1.152830,1.165857,150,gamble,-1.250,loss,-0.190,-0.750,22,DA039


## Set Conditions for TFRs

In [13]:
###### label main condition categories: regret/relief, choice, trial type [future - high/low CPE]
tfr_conditions = [
            # CPE - Regret vs Relief
            ['(data.decisionCPE<0)','(data.decisionCPE>0)'],
    
             # Choice - Gamble vs Safe
            ['(data.GambleChoice=="gamble")','(data.GambleChoice=="safe")'],
    
            # Low vs High BDI
            ['(data.bdi<20)','(data.bdi>20)'],
    
            # CPE x Choice - CPE + Gamble, CPE + Safe
            ['(data.decisionCPE<0) & (data.GambleChoice=="gamble")',
             '(data.decisionCPE>0) & (data.GambleChoice=="gamble")',
             '(data.decisionCPE<0) & (data.GambleChoice=="safe")',
             '(data.decisionCPE>0) & (data.GambleChoice=="safe")'],
    
            # CPE x BDI - CPE + Low BDI, CPE + High BDI
            ['(data.decisionCPE<0) & (data.bdi<20)',
             '(data.decisionCPE>0) & (data.bdi<20)',
             '(data.decisionCPE<0) & (data.bdi>20)',
             '(data.decisionCPE>0) & (data.bdi>20)'],
    
            # CPE x Choice x Low BDI - CPE+Gamble+Low BDI, CPE+Safe+Low BDI
            ['(data.decisionCPE<0) & (data.GambleChoice=="gamble") & (data.bdi<20)',
             '(data.decisionCPE>0) & (data.GambleChoice=="gamble") & (data.bdi<20)',
             '(data.decisionCPE<0) & (data.GambleChoice=="safe") & (data.bdi<20)',
             '(data.decisionCPE>0) & (data.GambleChoice=="safe") & (data.bdi<20)'],
    
            # CPE x Choice x High BDI - CPE+Gamble+high BDI, CPE+Safe+high BDI
            ['(data.decisionCPE<0) & (data.GambleChoice=="gamble") & (data.bdi>20)',
             '(data.decisionCPE>0) & (data.GambleChoice=="gamble") & (data.bdi>20)',
             '(data.decisionCPE<0) & (data.GambleChoice=="safe") & (data.bdi>20)',
             '(data.decisionCPE>0) & (data.GambleChoice=="safe") & (data.bdi>20)'] ]
#              # Trial Type - Loss, Mix, or Gain
#             ['(data.TrialType=="loss")','(data.TrialType=="mix")','(data.TrialType=="gain")'],

            # CPE x Choice x Loss
#             ['(data.decisionCPE<0) & (data.GambleChoice=="gamble") & (data.TrialType=="loss")',
#              '(data.decisionCPE>0) & (data.GambleChoice=="gamble") & (data.TrialType=="loss")',
#              '(data.decisionCPE<0) & (data.GambleChoice=="safe") & (data.TrialType=="loss")',
#              '(data.decisionCPE>0) & (data.GambleChoice=="safe") & (data.TrialType=="loss")'],
#             # CPE x Choice x Mix
#             ['(data.decisionCPE<0) & (data.GambleChoice=="gamble") & (data.TrialType=="mix")',
#              '(data.decisionCPE>0) & (data.GambleChoice=="gamble") & (data.TrialType=="mix")',
#              '(data.decisionCPE<0) & (data.GambleChoice=="safe") & (data.TrialType=="mix")',
#              '(data.decisionCPE>0) & (data.GambleChoice=="safe") & (data.TrialType=="mix")'],
#             # CPE x Choice x Gain
#             ['(data.decisionCPE<0) & (data.GambleChoice=="gamble") & (data.TrialType=="gain")',
#              '(data.decisionCPE>0) & (data.GambleChoice=="gamble") & (data.TrialType=="gain")',
#              '(data.decisionCPE<0) & (data.GambleChoice=="safe") & (data.TrialType=="gain")',
#              '(data.decisionCPE>0) & (data.GambleChoice=="safe") & (data.TrialType=="gain")']
# ]

  # High vs Low CPE
#             ['(np.abs(data.decisionCPE)>np.mean(np.abs(data.decisionCPE))',
#              '(np.abs(data.decisionCPE)<np.mean(np.abs(data.decisionCPE))'], 
# future - consider high/low CPE by *magnitude* - take abs value of CPE, take mean & threshold above/below


In [14]:
condition_labels = ['CPE','Choice','BDI','CPE_Choice','CPE_BDI','CPE_Choice_lowBDI','CPE_Choice_highBDI']
condition_master_key = {f'{cond}':tfr_conditions[ix] for ix,cond in enumerate(condition_labels)}
condition_master_key


{'CPE': ['(data.decisionCPE<0)', '(data.decisionCPE>0)'],
 'Choice': ['(data.GambleChoice=="gamble")', '(data.GambleChoice=="safe")'],
 'BDI': ['(data.bdi<20)', '(data.bdi>20)'],
 'CPE_Choice': ['(data.decisionCPE<0) & (data.GambleChoice=="gamble")',
  '(data.decisionCPE>0) & (data.GambleChoice=="gamble")',
  '(data.decisionCPE<0) & (data.GambleChoice=="safe")',
  '(data.decisionCPE>0) & (data.GambleChoice=="safe")'],
 'CPE_BDI': ['(data.decisionCPE<0) & (data.bdi<20)',
  '(data.decisionCPE>0) & (data.bdi<20)',
  '(data.decisionCPE<0) & (data.bdi>20)',
  '(data.decisionCPE>0) & (data.bdi>20)'],
 'CPE_Choice_lowBDI': ['(data.decisionCPE<0) & (data.GambleChoice=="gamble") & (data.bdi<20)',
  '(data.decisionCPE>0) & (data.GambleChoice=="gamble") & (data.bdi<20)',
  '(data.decisionCPE<0) & (data.GambleChoice=="safe") & (data.bdi<20)',
  '(data.decisionCPE>0) & (data.GambleChoice=="safe") & (data.bdi<20)'],
 'CPE_Choice_highBDI': ['(data.decisionCPE<0) & (data.GambleChoice=="gamble") & (dat

In [18]:
drop_cond = ['BDI','CPE_BDI','CPE_Choice_lowBDI','CPE_Choice_highBDI']

In [19]:
single_subj_key  = condition_master_key.copy()
single_subj_key  = {f'{key}':single_subj_key[key]  for key in list(single_subj_key.keys()) if key not in drop_cond}
single_subj_key

{'CPE': ['(data.decisionCPE<0)', '(data.decisionCPE>0)'],
 'Choice': ['(data.GambleChoice=="gamble")', '(data.GambleChoice=="safe")'],
 'CPE_Choice': ['(data.decisionCPE<0) & (data.GambleChoice=="gamble")',
  '(data.decisionCPE>0) & (data.GambleChoice=="gamble")',
  '(data.decisionCPE<0) & (data.GambleChoice=="safe")',
  '(data.decisionCPE>0) & (data.GambleChoice=="safe")']}

# Within-Subject ROI TFRs

In [15]:
# figsize_guide = [(18,6),(22,6),(18,12)]
figsize_guide = [(18,6),(18,12)]

In [20]:

for subj_id in subj_ids:
    print('Starting subj: ',subj_id)
    # subset subj behav df
    data = all_behav[all_behav.subj_id == subj_id]
    # find which ROIs from roi_list are in subj data
    subj_rois = [roi for roi in roi_list if roi in 
                 roi_reref_labels[roi_reref_labels.subj_id == subj_id].roi.unique().tolist()]
    # make dict with subj rois as keys and subj_roi channels as 
    subj_roi_elecs = {f'{roi}':roi_reref_labels.reref_ch_names[(roi_reref_labels.subj_id == subj_id) &
                                (roi_reref_labels.roi == roi)].values.tolist() for roi in subj_rois}
    
   
    subj_all_elecs = []
    for elec_list in subj_roi_elecs.values():
        subj_all_elecs.extend(elec_list)
    
    # load subj tfr data + extract roi channels
    subj_tfr_data = mne.time_frequency.read_tfrs(f'{neural_dir}{subj_id}/{subj_id}_CpeOnset-tfr.h5'
                                                )[0].pick_channels(subj_all_elecs)
        
    for roi in subj_rois:
        print('Starting roi: ',subj_id,roi)
        # count num elecs in ROI 
        num_elec = len(subj_roi_elecs[roi])
        # load subj power epochs data for roi channels  
        subj_roi_data = subj_tfr_data.copy().pick_channels(subj_roi_elecs[roi])._data
#         subj_roi_data = mne.time_frequency.read_tfrs(f'{neural_dir}{subj_id}/{subj_id}_CpeOnset-tfr.h5')[0].copy(
#         ).pick_channels(subj_roi_elecs[roi])._data
        
        for key in list(single_subj_key.keys()):            
            # empty list to store condition data for plot
            X = []
            for cond in single_subj_key[key]:
                # get indices for condition labels 
                cond_rounds = (data[eval(cond)].Round - 1).tolist()
                # extract mean condition specific tfr data across elecs in roi 
#                 cond_data = np.nanmean(np.nanmean(subj_roi_data[cond_rounds,:,:,:],axis=0),axis=0)
                X.append(np.nanmean(np.nanmean(subj_roi_data[cond_rounds,:,:,:],axis=0),axis=0))
                
            ### plot within subj ROI tfr by condition 
            # make fig directory 
            subj_roi_fig_dir = f'{base_dir}ephys_analysis/figs/roi_tfrs/single_subj/{roi}/{key}/'
            os.makedirs(subj_roi_fig_dir,exist_ok=True)
            
            # create empty plot with desired subplots 
            num_plots = len(single_subj_key[key]) #num conditions 
            # hacky way to get subplot layout
            nrow = int(num_plots/2) 
            ncol = int(np.round(np.round(num_plots/2)+(2/(num_plots))))
            
            figsize = figsize_guide[nrow-1]
            fig, ax = plt.subplots(nrow, ncol, figsize=figsize, dpi=300,constrained_layout=True)
            
            vmax = 0.25
            vmin = -0.25
            
            for tfr_ix,tfr_axes in enumerate(ax.flatten()):
                cond_name  = single_subj_key[key][tfr_ix]
                cond_title = (" ").join([c.split('.')[1].strip()[:-1].replace('"','').split(
                    '=')[-1].replace('decision','') for c 
                            in cond_name.split('&')])

                im = tfr_axes.imshow(X[tfr_ix],
                                    interpolation='Bicubic',
                                    aspect='auto', origin='lower', 
                                    cmap='RdBu_r',vmin = vmin, vmax = vmax)

                tfr_axes.set(xlabel='Time (s)', ylabel='Frequency', 
                             # extract conditions from key
                            title=cond_title,
                            yticks=np.arange(0,len(freqs))[::5], 
                            yticklabels=freqs[::5].astype(int), 
                            xticks = np.arange(0,len(times))[::500],
                            xticklabels= times[::500])

                fig.colorbar(im, ax=tfr_axes)
            
            fig.suptitle(f'{roi} {subj_id} n_elec={num_elec}')
                        
            plt.savefig(f'{subj_roi_fig_dir}{subj_id}_{roi}_{key}.pdf',dpi=300,metadata=None,
                        format='pdf',bbox_inches='tight',pad_inches=0.1,
                        facecolor='auto', edgecolor='auto',backend=None)
            plt.close()
            
            del X, fig, ax
        
        del subj_roi_data
            
    del subj_tfr_data
        
        

Starting subj:  MS002
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS002/MS002_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting roi:  MS002 dmpfc
Starting roi:  MS002 ofc
Starting roi:  MS002 acc
Starting roi:  MS002 sts
Starting roi:  MS002 hpc
Starting roi:  MS002 amy
Starting roi:  MS002 dlpfc
Starting roi:  MS002 vlpfc
Starting roi:  MS002 ains
Starting roi:  MS002 pins
Starting roi:  MS002 mtg
Starting roi:  MS002 motor
Starting subj:  MS003
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS003/MS003_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting roi:  MS003 dmpfc
Starting roi:  MS003 ofc
Starting roi:  MS003 acc
Starting roi:  MS003 sts
Starting roi:  MS003 amy
Starting roi:  MS003 dlpfc
Starting roi:  MS003 vlpfc
Starting roi:  MS003 ains
Starting roi:  MS003 pins
Starting roi:  MS003 mtg
Starting roi:  MS003 motor
Starting roi:  MS003 vmpfc
Starting subj:  MS009
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/dat

Starting roi:  MS033 mcc
Starting roi:  MS033 phg
Starting roi:  MS033 parietal
Starting subj:  MS035
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS035/MS035_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting roi:  MS035 dmpfc
Starting roi:  MS035 ofc
Starting roi:  MS035 acc
Starting roi:  MS035 sts
Starting roi:  MS035 hpc
Starting roi:  MS035 amy
Starting roi:  MS035 dlpfc
Starting roi:  MS035 stg
Starting roi:  MS035 vlpfc
Starting roi:  MS035 ains
Starting roi:  MS035 smg
Starting roi:  MS035 pins
Starting roi:  MS035 thalamus
Starting subj:  MS041
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS041/MS041_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting roi:  MS041 dmpfc
Starting roi:  MS041 ofc
Starting roi:  MS041 acc
Starting roi:  MS041 sts
Starting roi:  MS041 hpc
Starting roi:  MS041 amy
Starting roi:  MS041 dlpfc
Starting roi:  MS041 stg
Starting roi:  MS041 vlpfc
Starting roi:  MS041 ains
Starting roi:  MS041 pins
Sta

# All Subject ROI TFRs

In [20]:
# figsize_ guide = [(18,6),(22,6),(18,12)]

In [17]:

for roi in roi_list: 
    print('Starting roi: ',roi)
    # list of subj_ids with contacts in roi
    roi_subj_ids = roi_reref_labels.subj_id[roi_reref_labels.roi == roi].unique().tolist()
    # make dict of subj_id keys and list of channels in roi 
    roi_subj_elecs = {f'{subj_id}':roi_reref_labels.reref_ch_names[(roi_reref_labels.roi == roi)&
                                (roi_reref_labels.subj_id==subj_id)].unique().tolist() for subj_id in roi_subj_ids}
    
    # dict to store data 
#     roi_cond_data = {f'{key}':[] for key,value in group_conditions.items()}
    
    for key in list(condition_master_key.keys()):
        print('Starting Condition for ROI: ',key,roi)
        
        # create empty data vec for condition data
        roi_X = {f'{cond_key}':[] for cond_key in condition_master_key[key]}
        
        for subj_id in roi_subj_elecs.keys():
            print('Starting subj for ROI: ',subj_id,roi)
            
            # extract subj behav data 
            data = all_behav[all_behav.subj_id == subj_id]
            
            # load single subject tfr data + extract ROI channels 
            subj_roi_data = mne.time_frequency.read_tfrs(
                f'{neural_dir}{subj_id}/{subj_id}_CpeOnset-tfr.h5')[0].copy().pick_channels(
                roi_subj_elecs[subj_id])._data
         
            # iterate through individual conditions in each condition type 
            for cond_key in condition_master_key[key]:
                # find subj round numbers for each condition 
                
                # get indices for condition labels 
                cond_rounds = (data[eval(cond_key)].Round - 1).tolist()
                
                # extract mean condition specific tfr data across elecs in roi
                
                roi_X[cond_key].append(np.nanmean(np.nanmean(subj_roi_data[cond_rounds,:,:,:],axis=0),axis=0))
            
            del data,subj_roi_data
                
        # Global condition data needs to be averaged - one key across subj
        
        # X = list of freqxtime arrays averaged across subj for each plot condition within key
        X = [np.nanmean(np.stack([np.array(data) for data in roi_X[cond_id]]),axis=0) for 
             cond_id in roi_X.keys()]
        

        ### plot within subj ROI tfr by condition 
        # make fig directory 
        roi_fig_dir = f'{base_dir}ephys_analysis/figs/roi_tfrs/subj_agg/{key}/'
        os.makedirs(roi_fig_dir,exist_ok=True)
        
        # get len of all ch in roi 
        num_roi_elecs = np.sum([len(elec_list) for elec_list in roi_subj_elecs.values()])
        # create empty plot with desired subplots 
        num_plots = len(X) #num conditions 
        # hacky way to get subplot layout
        nrow = int(num_plots/2) 
        ncol = int(np.round(np.round(num_plots/2)+(2/(num_plots))))

        figsize = figsize_guide[nrow-1]
        fig, ax = plt.subplots(nrow, ncol, figsize=figsize, dpi=300,constrained_layout=True)

        vmax = 0.1
        vmin = -0.1

        for tfr_ix,tfr_axes in enumerate(ax.flatten()):
            
            cond_name  = condition_master_key[key][tfr_ix]
            cond_title = (" ").join([c.split('.')[1].strip()[:-1].replace('"','').split('=')[-1].replace(
                'decision','') for c in cond_name.split('&')])

            im = tfr_axes.imshow(X[tfr_ix],
                                interpolation='Bicubic',
                                aspect='auto', origin='lower', 
                                cmap='RdBu_r',vmin = vmin, vmax = vmax)

            tfr_axes.set(xlabel='time from cpe (s)', ylabel='frequency (Hz)', 
                         # extract conditions from key
                        title=cond_title,
                        yticks=np.arange(0,len(freqs))[::5], 
                        yticklabels=freqs[::5].astype(int), 
                        xticks = np.arange(0,len(times))[::500],
                        xticklabels= times[::500])

            fig.colorbar(im, ax=tfr_axes)

        fig.suptitle(f'{roi} {key} n_subj={len(roi_subj_ids)} n_elec={num_roi_elecs}')

        plt.savefig(f'{roi_fig_dir}{roi}_{key}_nelec-{num_roi_elecs}.pdf',dpi=300,metadata=None,
                    format='pdf',bbox_inches='tight',pad_inches=0.1,
                    facecolor='auto', edgecolor='auto',backend=None)
        plt.close()

        del X, fig, ax


        
        
        
#     

Starting roi:  dmpfc
Starting Condition for ROI:  CPE dmpfc
Starting subj for ROI:  MS002 dmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS002/MS002_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS003 dmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS003/MS003_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS009 dmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS009/MS009_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS011 dmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS011/MS011_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS015 dmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS015/MS015_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS016 dmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS016/MS016_CpeOnset-tfr.h5 ...


Adding metadata with 19 columns
Starting subj for ROI:  MS015 dmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS015/MS015_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS016 dmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS016/MS016_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS017 dmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS017/MS017_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS019 dmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS019/MS019_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS022 dmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS022/MS022_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS024 dmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS024/MS024_CpeOnset-tfr.h5 ...
Adding metadata with 19 colu

Starting subj for ROI:  MS022 dmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS022/MS022_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS024 dmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS024/MS024_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS025 dmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS025/MS025_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS026 dmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS026/MS026_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS027 dmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS027/MS027_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS028 dmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS028/MS028_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS02

Starting subj for ROI:  MS027 dmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS027/MS027_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS028 dmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS028/MS028_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS029 dmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS029/MS029_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS030 dmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS030/MS030_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS033 dmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS033/MS033_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS035 dmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS035/MS035_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS04

Adding metadata with 19 columns
Starting subj for ROI:  MS041 ofc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS041/MS041_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS043 ofc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS043/MS043_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS048 ofc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS048/MS048_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS050 ofc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS050/MS050_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  DA037 ofc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/DA037/DA037_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  DA039 ofc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/DA039/DA039_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting

...

Adding metadata with 19 columns
Starting subj for ROI:  MS030 motor
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS030/MS030_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  DA037 motor
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/DA037/DA037_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  DA039 motor
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/DA039/DA039_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting Condition for ROI:  CPE_BDI motor
Starting subj for ROI:  MS002 motor
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS002/MS002_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS003 motor
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS003/MS003_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS009 motor
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS009/MS009_CpeOn

Adding metadata with 19 columns
Starting subj for ROI:  DA8 phg
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/DA8/DA8_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  DA026 phg
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/DA026/DA026_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting Condition for ROI:  BDI phg
Starting subj for ROI:  MS009 phg
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS009/MS009_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS011 phg
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS011/MS011_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS024 phg
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS024/MS024_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS025 phg
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS025/MS025_CpeOnset-tfr.h5 ...
Adding me

Adding metadata with 19 columns
Starting Condition for ROI:  Choice parietal
Starting subj for ROI:  MS016 parietal
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS016/MS016_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS019 parietal
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS019/MS019_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS026 parietal
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS026/MS026_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS027 parietal
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS027/MS027_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS030 parietal
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS030/MS030_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS033 parietal
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/da

Adding metadata with 19 columns
Starting roi:  vmpfc
Starting Condition for ROI:  CPE vmpfc
Starting subj for ROI:  MS003 vmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS003/MS003_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS017 vmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS017/MS017_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS019 vmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS019/MS019_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS025 vmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS025/MS025_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS029 vmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS029/MS029_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Starting subj for ROI:  MS050 vmpfc
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/

Adding metadata with 19 columns


In [ ]:
##### power line plots

# for region in rois: 
#     # if region == 'TC':
#     #     region = (('Temporal') | ('Temporal Pole') | ('STS'))
#     fig, ax = plt.subplots(1, 1, figsize=(10, 6), dpi=300, sharey=True, sharex=True)
#     for ix, cond in enumerate(conditions): 
#         # Set the times for 
#         n_elecs = np.squeeze(np.stack(tfr_group_data[region][cond])).shape[0]
#         times = np.linspace(0, 1.5, np.squeeze(np.stack(tfr_group_data[region][cond])).shape[2])

# #         if cond_name == 'SME':
# #             plot_data = np.nanmedian(np.nanmedian(np.squeeze(np.stack(tfr_group_data[region][cond])), axis=0), axis=0)
# #         else:
#         plot_data = np.nanmean(np.squeeze(np.stack(tfr_group_data[region][cond])), axis=0)
#         ax.plot(times, np.nanmean(plot_data[freqs<9, :], axis=0))
      
#         ax.set(xlabel='Time (s)', ylabel='Power (z)', title=f'{cond}_{region}_nelec={n_elecs}')
#         # fig.colorbar(im, ax=ax[ix])
#         plt.savefig(f'/sc/arion/work/qasims01/MemoryBanditData/EMU/plots/feedback_theta_{cond_name}_{region}.pdf', dpi=300)
